In [1]:
# IMPORTING LIBRARIES 
import pandas as pd
import sklearn.preprocessing
from sklearn.neural_network import MLPClassifier 

In [60]:
# READ IN DATA
df_second_ten = pd.read_csv("data/EUR_GBP_2015_2005.csv")
df_first_ten = pd.read_csv("data/EUR_GBP_2025_2015.csv")

df = pd.concat([df_second_ten, df_first_ten], ignore_index = True)

df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%y")

df = df.sort_values(by =  "Date", ascending=True)

df = df.rename(columns = {'Price': 'Close'})
df = df[['Date', 'Close', 'Open', 'High', 'Low', 'Change %']]
df = df[['Date', 'Close', 'Open', 'High', 'Low', 'Change %']].copy()
df.head()

,Date,Close,Open,High,Low,Change %
2629,2005-09-16,0.6765,0.6768,0.6790,0.6746,0.00%
2628,2005-09-19,0.6735,0.6747,0.6759,0.6723,-0.44%
2627,2005-09-20,0.6736,0.6734,0.6753,0.6725,0.01%
2626,2005-09-21,0.6750,0.6737,0.6763,0.6727,0.21%
2625,2005-09-22,0.6786,0.6750,0.6800,0.6743,0.53%


In [61]:
# CREATE PRIOR n DAYS FEATURE

n = 3 # starting here

for before in range(1, n+1):
    df[f"Close_{before}_before"] = df["Close"].shift(before)
    df[f"Open_{before}_before"] = df["Open"].shift(before)
    df[f"High_{before}_before"] = df["High"].shift(before)
    df[f"Low_{before}_before"] = df["Low"].shift(before)

df = df.dropna()
df = df.drop(columns = "Date")
df.head()

,Close,Open,High,Low,Change %,Close_1_before,Open_1_before,High_1_before,Low_1_before,Close_2_before,Open_2_before,High_2_before,Low_2_before,Close_3_before,Open_3_before,High_3_before,Low_3_before
2626,0.6750,0.6737,0.6763,0.6727,0.21%,0.6736,0.6734,0.6753,0.6725,0.6735,0.6747,0.6759,0.6723,0.6765,0.6768,0.6790,0.6746
2625,0.6786,0.6750,0.6800,0.6743,0.53%,0.6750,0.6737,0.6763,0.6727,0.6736,0.6734,0.6753,0.6725,0.6735,0.6747,0.6759,0.6723
2624,0.6778,0.6788,0.6801,0.6769,-0.12%,0.6786,0.6750,0.6800,0.6743,0.6750,0.6737,0.6763,0.6727,0.6736,0.6734,0.6753,0.6725
2623,0.6787,0.6791,0.6801,0.6774,0.13%,0.6778,0.6788,0.6801,0.6769,0.6786,0.6750,0.6800,0.6743,0.6750,0.6737,0.6763,0.6727
2622,0.6799,0.6785,0.6809,0.6781,0.18%,0.6787,0.6791,0.6801,0.6774,0.6778,0.6788,0.6801,0.6769,0.6786,0.6750,0.6800,0.6743
